# Imports

In [1]:
import pandas as pd
import numpy as np

import os
import sys

# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

# import keras
# from keras.callbacks import ReduceLROnPlateau
# from keras.models import Sequential
# from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
# from keras.utils import np_utils, to_categorical
# from keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
github_csv_url = r"C:/Users/gusta/Documents/Data Science Bootcamp/Data Science/Final Project/Data/completeAudioSetdf.csv"

data_path = pd.read_csv(github_csv_url)



### Data Aug Function

In [34]:
# NOISE INJECTION
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

# TIME STRETCHING
def stretch(data, rate):
    return librosa.effects.time_stretch(data, rate=0.8)

# TIME SHIFTING
def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*35000)
    return np.roll(data, shift_range)

# PITCH SHIFTING
def pitch(data, sampling_rate, pitch_factor=3):
    return librosa.effects.pitch_shift(data, sr=sampling_rate, n_steps=pitch_factor)


## Feature Extraction

The following feature selection will be used:

* Zero Crossing Rate
* Chroma_stft
* MFCC
* RMS(root mean square) value
* MelSpectogram to train our model.

### Functions

Extract features function

In [47]:
def extract_features(data):
    # Zero Crossing Rate
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result = np.hstack((result, zcr))  # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft).T, axis=0)
    result = np.hstack((result, chroma_stft))  # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data).T, axis=0)
    result = np.hstack((result, mfcc))  # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms))  # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data).T, axis=0)
    result = np.hstack((result, mel))  # stacking horizontally

    return result

Get features function

In [49]:


def get_features(path):
    # duration and offset are used to take care of the no audio at the start and end of each audio file as seen above.
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    
    # without augmentation
    res1 = extract_features(data)
    result = np.array(res1)
    
    # data with noise
    noise_data = noise(data)
    res2 = extract_features(noise_data)
    result = np.vstack((result, res2))  # stacking vertically
    
    # data with stretching and pitching
    new_data = stretch(data, rate=0.8)
    data_stretch_pitch = pitch(new_data, sample_rate)
    res3 = extract_features(data_stretch_pitch)
    result = np.vstack((result, res3))  # stacking vertically
    
    return result

### Using the functions to get convert the audios into a dataframe

In [ ]:
X, Y = [], []
for path, emotion in zip(data_path.Path, data_path.Emotions):
    feature = get_features(path)
    for ele in feature:
        X.append(ele)
        # appending emotion 3 times as we have made 3 augmentation techniques on each audio file.
        Y.append(emotion)

In [43]:
len(X), len(Y), data_path.Path.shape

(36486, 36486, (12162,))

In [45]:
Features = pd.DataFrame(X)
Features['labels'] = Y
#Features.to_csv('C:/Users/gusta/Documents/Data Science Bootcamp/Data Science/Final Project/Data/features.csv', index=False)
Features.head()

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,labels
0,0.321275,0.729664,0.750032,0.730624,0.735275,0.713529,0.660531,0.684966,0.733049,0.753972,...,4.310903e-06,0.000003,0.000002,2.279739e-06,5.116493e-06,8.190282e-06,0.000007,5.051210e-06,4.245834e-07,neutral
1,0.325693,0.782074,0.826724,0.822661,0.815199,0.818640,0.718994,0.672557,0.718188,0.771710,...,7.605765e-05,0.000075,0.000076,7.231593e-05,7.378907e-05,7.971013e-05,0.000083,7.776085e-05,7.396648e-05,neutral
2,0.195482,0.786348,0.712587,0.687922,0.678074,0.722655,0.658071,0.629409,0.654643,0.664657,...,8.940135e-07,0.000001,0.000001,6.153283e-07,5.278833e-07,6.426405e-07,0.000001,4.702370e-07,2.483089e-08,neutral
3,0.293566,0.673896,0.722096,0.723508,0.682302,0.680533,0.675352,0.628977,0.679179,0.707283,...,6.984504e-06,0.000007,0.000007,6.979548e-06,1.214236e-05,9.640183e-06,0.000011,5.543237e-06,4.254087e-07,neutral
4,0.334002,0.806489,0.840281,0.804711,0.795291,0.807813,0.716666,0.669026,0.729889,0.756993,...,1.468454e-04,0.000147,0.000150,1.453142e-04,1.483842e-04,1.537717e-04,0.000158,1.487933e-04,1.363863e-04,neutral
